In [1]:
from os import listdir
from os.path import isfile, join
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
from nltk.tokenize import word_tokenize
import pickle
from gensim.similarities import MatrixSimilarity
from gensim.summarization.summarizer import summarize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ronnald/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
with open('label_content', 'rb') as f:
    labelContent = pickle.load(f)

In [3]:
tagged_data = [TaggedDocument(words=word_tokenize(content.lower()), tags=[str(label)]) for label, content in labelContent.items()]

In [4]:
tagged_data

[TaggedDocument(words=['ue', 'isso', 'se', 'tornou', 'motivo', 'de', 'existencia', 'em', 'muitas', 'culturas', 'quando', 'se', 'fala', 'na', 'etimologia', 'da', 'palavra', 'religiao', 'percebe', 'se', 'a', 'importancia', 'para', 'todos', 'de', 'se', 'ligar', 'conectar', 'com', 'algo', 'maior', 'uma', 'forca', 'o', 'divino', 'e', 'o', 'quanto', 'isso', 'e', 'fundamental', 'para', 'tantas', 'pessoas', 'saber', 'que', 'existe', 'algo', 'por', 'tras', 'de', 'todo', 'o', 'acaso', 'do', 'mundo', 'e', 'existe', 'uma', 'razao', 'maior', 'do', 'que', 'tudo', 'na', 'vida', 'no', 'cenario', 'brasileiro', 'o', 'cristianismo', 'parece', 'ser', 'a', 'religiao', 'com', 'a', 'maior', 'pluralidade', 'de', 'vertentes', 'do', 'pentecostalismo', 'ao', 'protestantismo', 'tradicional', 'existe', 'uma', 'grande', 'variedade', 'de', 'denominacoes', 'que', 'interagem', 'entre', 'si', 'dessa', 'forma', 'as', 'religioes', 'pentecostais', 'elaboram', 'um', 'discurso', 'de', 'supremacia', 'que', 'por', 'consequenc

In [5]:
max_epochs = 10
vec_size = 100#neurons
alpha = 0.025
window = 3

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1,
                window = window)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    #model.min_alpha = model.alpha

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


In [71]:
def parseRelations(doc,most_similar):
    #Get top 5
    docRelation[doc] = {}
    for i in range(0,5):
        doc2,rel = most_similar[i]
        docRelation[doc][doc2]={'weight':rel}

In [77]:
#vou transformar docReletion em um dicionario. Melhor para o networkx
docRelation ={}
for i in range(0,len(tagged_data)):
    if(model.docvecs.most_similar(i)[0][1] > 0.65):
        parseRelations(str(tagged_data[i][1][0]),model.docvecs.most_similar(i))
        
with open('graph', 'wb') as f:
        pickle.dump(docRelation, f)
docRelation

/home/ronnald/dev/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


{'2017-1-181805': {'2018-1-300633': {'weight': 0.6564905047416687},
  '2017-2-257015': {'weight': 0.611336886882782},
  '2017-1-182231': {'weight': 0.6014813184738159},
  '2018-2-345397': {'weight': 0.5849880576133728},
  '2018-2-346128': {'weight': 0.5667991042137146}},
 '2017-1-181810': {'2018-2-346133': {'weight': 0.6628072261810303},
  '2018-1-301158': {'weight': 0.5701789855957031},
  '2017-1-182967': {'weight': 0.5589055418968201},
  '2017-2-261440': {'weight': 0.5418500900268555},
  '2018-1-301382': {'weight': 0.5363973379135132}},
 '2017-1-182262': {'2017-2-261435': {'weight': 0.9476767182350159},
  '2018-2-349602': {'weight': 0.598151445388794},
  '2018-1-302064': {'weight': 0.5920310020446777},
  '2017-2-257028': {'weight': 0.5793975591659546},
  '2018-1-299276': {'weight': 0.5619028806686401}},
 '2017-1-181141': {'2017-2-257000': {'weight': 0.8361032009124756},
  '2018-2-350190': {'weight': 0.5920007228851318},
  '2018-1-301158': {'weight': 0.591431736946106},
  '2017-2-2557

In [7]:
model.wv.wmdistance(document1='2017-2-261435',document2='2017-1-182262')

2.506652095319885

In [13]:
model.wv.wmdistance(document1='2017-2-257028',document2='2017-1-182262')
#https://stackoverflow.com/questions/39455741/gcc-error-trying-to-exec-cc1plus-execvp-no-such-file-or-directory

2.144769657465015

In [16]:
with open('label_sentence', 'rb') as f:
    labelSentence = pickle.load(f)

In [17]:
print(summarize(labelSentence["2017-1-182262"],ratio=0.2))

Infelizmente no Brasil e no mundo, muitas vezes as pessoas esquecem os valores da religiao e tem um preconceito baseado na sociedade, assim gerando a intolerancia e ate a violencia religiosa.
Estas diferencas acabam gerando uma intolerancia, pois as pessoas tendem a pensar que o que elas acreditam e o certo, assim criam um preconceito contra pessoas de religioes diferentes de sua propria.
No brasil infelizmente existe a intolerancia religiosa; podemos ver um exemplo disto com alguns pastores evangelicos que demonizam as religioes da umbanda e de origem africana.
Podemos analisar isto e ver que mesmo com a maioria da populacao sendo catolica as pessoas vem mudando de religiao.
Tambem acredito ser de extrema importancia o dialogo inter-religioso, evitando que uma religiao condene a outra; como acontece no Brasil de alguns grupos evangelicos contra as religioes africanas.


In [18]:
print(summarize(labelSentence["2017-2-261435"],ratio=0.2))

Infelizmente no Brasil e no mundo, muitas vezes as pessoas esquecem os valores da religiao e tem um preconceito baseado na sociedade, assim gerando a intolerancia e ate a violencia religiosa.
Estas diferencas acabam gerando uma intolerancia, pois as pessoas tendem a pensar que o que elas acreditam e o certo, assim criam um preconceito contra pessoas de religioes diferentes de sua propria.
No Brasil infelizmente existe a intolerancia religiosa; podemos ver um exemplo disto com alguns pastores evangelicos que demonizam as religioes da umbanda e de origem africana.
Tambem acredito ser de extrema importancia o dialogo inter-religioso, evitando que uma religiao condene a outra; como acontece no Brasil de alguns grupos evangelicos contra as religioes africanas.
